In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report

# # required for class labels (colab default version is 0.14)
!sudo pip install mlxtend --upgrade --no-deps
from mlxtend.plotting import plot_confusion_matrix

# load the traces for gen3, gen4, gen8 and gen10
!wget https://github.com/drawnapart/drawnapart/raw/master/gen3_onscreen_mul_cpu_timer.tgz
!wget https://github.com/drawnapart/drawnapart/raw/master/gen3_onscreen_exp2_cpu_timer.tgz

!wget https://github.com/drawnapart/drawnapart/raw/master/gen4_offscreen_mul_cpu_timer.tgz
!wget https://github.com/drawnapart/drawnapart/raw/master/gen4_offscreen_sinh_cpu_timer.tgz

!wget https://github.com/drawnapart/drawnapart/raw/master/gen8_offscreen_sinh_cpu_timer.tgz
!wget https://github.com/drawnapart/drawnapart/raw/master/gen8_offscreen_sinh_gpu_timer.tgz

!wget https://github.com/drawnapart/drawnapart/raw/master/gen10_offscreen_sinh_cpu_timer.tgz
!wget https://github.com/drawnapart/drawnapart/raw/master/gen10_offscreen_sinh_gpu_timer.tgz

!tar xzvf gen3_onscreen_mul_cpu_timer.tgz
!tar xzvf gen3_onscreen_exp2_cpu_timer.tgz

!tar xzvf gen4_offscreen_mul_cpu_timer.tgz
!tar xzvf gen4_offscreen_sinh_cpu_timer.tgz

!tar xzvf gen8_offscreen_sinh_cpu_timer.tgz
!tar xzvf gen8_offscreen_sinh_gpu_timer.tgz

!tar xzvf gen10_offscreen_sinh_cpu_timer.tgz
!tar xzvf gen10_offscreen_sinh_gpu_timer.tgz

gen3_onscreen_mul_cpu_timer_traces = pd.read_csv('gen3_onscreen_mul_cpu_timer.csv')
gen3_onscreen_exp2_cpu_timer_traces = pd.read_csv('gen3_onscreen_exp2_cpu_timer.csv')

gen4_offscreen_mul_cpu_timer_traces = pd.read_csv('gen4_offscreen_mul_cpu_timer.csv')
gen4_offscreen_sinh_cpu_timer_traces = pd.read_csv('gen4_offscreen_sinh_cpu_timer.csv')

gen8_offscreen_sinh_cpu_timer_traces = pd.read_csv('gen8_offscreen_sinh_cpu_timer.csv')
gen8_offscreen_sinh_gpu_timer_traces = pd.read_csv('gen8_offscreen_sinh_gpu_timer.csv')

gen10_offscreen_sinh_cpu_timer_traces = pd.read_csv('gen10_offscreen_sinh_cpu_timer.csv')
gen10_offscreen_sinh_gpu_timer_traces = pd.read_csv('gen10_offscreen_sinh_gpu_timer.csv')

In [ ]:
gen4_offscreen_sinh_cpu_timer_traces.columns

In [ ]:
# Visualize some traces from the offscreen gen4 traces

number_of_labels = 4
labels_to_traces = {}
for label, traces in gen4_offscreen_sinh_cpu_timer_traces.groupby('label'):
  # take 4 random traces for this label
  labels_to_traces[label] = traces.sample(frac=1, random_state=19).iloc[:4, 1:]
  if len(labels_to_traces.keys()) >= number_of_labels:
    break

stacked = np.vstack(labels_to_traces.values())
stacked = np.log(stacked)
shared_max = np.max(stacked)
shared_min = np.min(stacked)

for i, (label, traces) in enumerate(labels_to_traces.items()):
  plt.figure(figsize=(10, 4))
  sns.heatmap(np.log(traces), cmap='nipy_spectral', vmin=shared_min, vmax=shared_max)
  plt.title(f'Visualization Of 4 Traces, device {i}')
  plt.xlabel('sample index')
  plt.ylabel('trace index')
  plt.xticks([])
  plt.yticks([])
  plt.show()

In [ ]:
def classify(dataset, name):
    print('--------------------------' + name + '--------------------------')
    # split into train and test
    y = dataset.iloc[:,0]
    X = dataset.iloc[:,1:]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

    # train and test the classifier
    clf = RandomForestClassifier(n_estimators=560)
    clf.fit(X_train, y_train)

    y_pred = clf.predict(X_test)
    print(classification_report(y_test, y_pred))

    # display the confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    fig, ax = plot_confusion_matrix(conf_mat=cm,
                                show_absolute=True,
                                show_normed=True,
                                colorbar=True,
                                class_names=np.unique(y_test),
                                figsize=(13,13),
                                cmap='Blues')
    plt.show()




In [ ]:
classify(gen3_onscreen_mul_cpu_timer_traces, 'gen3 mul cpu timer')
classify(gen3_onscreen_exp2_cpu_timer_traces, 'gen3 exp2 cpu timer')

classify(gen4_offscreen_mul_cpu_timer_traces, 'gen4 mul cpu timer')
classify(gen4_offscreen_sinh_cpu_timer_traces, 'gen4 sinh cpu timer')

classify(gen8_offscreen_sinh_cpu_timer_traces, 'gen8 sinh cpu timer')
classify(gen8_offscreen_sinh_gpu_timer_traces, 'gen8 sinh gpu timer')

classify(gen10_offscreen_sinh_cpu_timer_traces, 'gen10 sinh cpu timer')
classify(gen10_offscreen_sinh_gpu_timer_traces, 'gen10 sinh gpu timer')